# 安徽安庆市项目`WRF-CMAQ`模拟分析
## 模拟结果预处理：*`MET-Data`*

---
*@author: Evan*\
*@date: 2023-08-08*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd

import sys
sys.path.append('../../src/')
from namelist import *

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
isam = xr.open_dataset('D:/data/Project_Anqing/202307/COMBINE_ACONC_v54_ISAM_intel_CN3AH_135X138_2023_isam2.nc')
grid = xr.open_dataset('D:/data/Project_Anqing/GRIDCRO2D_2023141.nc')

In [3]:
# 将时间点转换为实际时间
times=pd.date_range(timestart,timeend,freq='h')
times

DatetimeIndex(['2023-07-01 00:00:00', '2023-07-01 01:00:00',
               '2023-07-01 02:00:00', '2023-07-01 03:00:00',
               '2023-07-01 04:00:00', '2023-07-01 05:00:00',
               '2023-07-01 06:00:00', '2023-07-01 07:00:00',
               '2023-07-01 08:00:00', '2023-07-01 09:00:00',
               ...
               '2023-07-31 14:00:00', '2023-07-31 15:00:00',
               '2023-07-31 16:00:00', '2023-07-31 17:00:00',
               '2023-07-31 18:00:00', '2023-07-31 19:00:00',
               '2023-07-31 20:00:00', '2023-07-31 21:00:00',
               '2023-07-31 22:00:00', '2023-07-31 23:00:00'],
              dtype='datetime64[ns]', length=744, freq='H')

In [4]:
# 将层数转换为气压高度
preslevel=np.array(
    [1.,     0.9979, 0.9956, 0.9931, 0.9904, 0.9875, 0.9844, 0.9807, 0.9763, 0.9711,
     0.9649, 0.9575, 0.9488, 0.9385, 0.9263, 0.912,  0.8951, 0.8753, 0.8521, 0.8251,
     0.7937, 0.7597, 0.7229, 0.6883, 0.641,  0.596,  0.5484, 0.4985, 0.4467, 0.3934,
     0.3393, 0.285,  0.2316, 0.1801, 0.1324, 0.0903, 0.0542, 0.0241,]
    )
pres=preslevel*950+50
pres

array([1000.   ,  998.005,  995.82 ,  993.445,  990.88 ,  988.125,
        985.18 ,  981.665,  977.485,  972.545,  966.655,  959.625,
        951.36 ,  941.575,  929.985,  916.4  ,  900.345,  881.535,
        859.495,  833.845,  804.015,  771.715,  736.755,  703.885,
        658.95 ,  616.2  ,  570.98 ,  523.575,  474.365,  423.73 ,
        372.335,  320.75 ,  270.02 ,  221.095,  175.78 ,  135.785,
        101.49 ,   72.895])

In [5]:
isam

<xarray.Dataset>
Dimensions:  (TSTEP: 768, VAR: 24, DATE-TIME: 2, LAY: 26, ROW: 138, COL: 135)
Dimensions without coordinates: TSTEP, VAR, DATE-TIME, LAY, ROW, COL
Data variables: (12/25)
    TFLAG    (TSTEP, VAR, DATE-TIME) int32 ...
    PMI_AnQ  (TSTEP, LAY, ROW, COL) float32 ...
    PMJ_AnQ  (TSTEP, LAY, ROW, COL) float32 ...
    PMK_AnQ  (TSTEP, LAY, ROW, COL) float32 ...
    PMI_AQI  (TSTEP, LAY, ROW, COL) float32 ...
    PMJ_AQI  (TSTEP, LAY, ROW, COL) float32 ...
    ...       ...
    PMI_ICO  (TSTEP, LAY, ROW, COL) float32 ...
    PMJ_ICO  (TSTEP, LAY, ROW, COL) float32 ...
    PMK_ICO  (TSTEP, LAY, ROW, COL) float32 ...
    PMI_BCO  (TSTEP, LAY, ROW, COL) float32 ...
    PMJ_BCO  (TSTEP, LAY, ROW, COL) float32 ...
    PMK_BCO  (TSTEP, LAY, ROW, COL) float32 ...
Attributes: (12/33)
    IOAPI_VERSION:  ioapi-3.2: $Id: init3.F90 185 2020-08-28 16:49:45Z coats ...
    EXEC_ID:        ????????????????                                         ...
    FTYPE:          1
    CDATE:          2023218
    CTIME:          233221
    WDATE:          2023218
    ...             ...
    VGLVLS:         [1.     0.9979 0.9956 0.9931 0.9904 0.9875 0.9844 0.9807 ...
    GDNAM:          CN3AH_135X138   
    UPNAM:          COMBINE         
    VAR-LIST:       PMI_AnQ         PMJ_AnQ         PMK_AnQ         PMI_AQI  ...
    FILEDESC:       Src Apportionment file output                            ...
    HISTORY:

In [6]:
days=1 # set spin-up days
dataset=xr.Dataset(
    data_vars=dict(
        # ! vars from ISAM
        PM_AnQ=(['time','level','y','x'],isam.PMI_AnQ[days*24-8:-8,:,:,:].data+isam.PMJ_AnQ[days*24-8:-8,:,:,:].data,{'long name':'PMI from Anqing','units':'ug m-3'}),
        PM_AQI=(['time','level','y','x'],isam.PMI_AQI[days*24-8:-8,:,:,:].data+isam.PMJ_AQI[days*24-8:-8,:,:,:].data,{'long name':'PMI from Anqing Industry','units':'ug m-3'}),
        PM_AQT=(['time','level','y','x'],isam.PMI_AQT[days*24-8:-8,:,:,:].data+isam.PMJ_AQT[days*24-8:-8,:,:,:].data,{'long name':'PMI from Anqing Transportation','units':'ug m-3'}),
        PM_AQA=(['time','level','y','x'],isam.PMI_AQA[days*24-8:-8,:,:,:].data+isam.PMJ_AQA[days*24-8:-8,:,:,:].data,{'long name':'PMI from Anqing Agriculture','units':'ug m-3'}),
        PM_AQP=(['time','level','y','x'],isam.PMI_AQP[days*24-8:-8,:,:,:].data+isam.PMJ_AQP[days*24-8:-8,:,:,:].data,{'long name':'PMI from Anqing Power','units':'ug m-3'}),
        PM_OTH=(['time','level','y','x'],isam.PMI_OTH[days*24-8:-8,:,:,:].data+isam.PMJ_OTH[days*24-8:-8,:,:,:].data,{'long name':'PMI from OTHER','units':'ug m-3'}),
        PM_ICO=(['time','level','y','x'],isam.PMI_ICO[days*24-8:-8,:,:,:].data+isam.PMJ_ICO[days*24-8:-8,:,:,:].data,{'long name':'PMI from ICON','units':'ug m-3'}),
        PM_BCO=(['time','level','y','x'],isam.PMI_BCO[days*24-8:-8,:,:,:].data+isam.PMJ_BCO[days*24-8:-8,:,:,:].data,{'long name':'PMI from BCON','units':'ug m-3'}),
        ),
    coords=dict(
        time=times,
        level=pres[:26],
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Anqing_202307',
        grid='CN3AH_135X138',
        createtime=pd.Timestamp.now().strftime('%Y-%m-%d'),
    ),
)
dataset

<xarray.Dataset>
Dimensions:    (time: 744, level: 26, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2023-07-01 ... 2023-07-31T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 703.9 659.0 616.2
    latitude   (y, x) float32 29.01 29.01 29.01 29.01 ... 32.7 32.7 32.7 32.7
    longitude  (y, x) float32 114.7 114.7 114.7 114.8 ... 119.0 119.0 119.1
Dimensions without coordinates: y, x
Data variables:
    PM_AnQ     (time, level, y, x) float32 3.006e-11 1.086e-11 ... 5.453e-21
    PM_AQI     (time, level, y, x) float32 3.006e-11 1.086e-11 ... 5.453e-21
    PM_AQT     (time, level, y, x) float32 3.006e-11 1.086e-11 ... 5.453e-21
    PM_AQA     (time, level, y, x) float32 3.006e-11 1.086e-11 ... 5.453e-21
    PM_AQP     (time, level, y, x) float32 3.006e-11 1.086e-11 ... 5.453e-21
    PM_OTH     (time, level, y, x) float32 0.3348 0.08877 ... 5.998e-09
    PM_ICO     (time, level, y, x) float32 3.006e-11 1.086e-11 ... 5.453e-21
    PM_BCO     (time, level, y, x) float32 16.62 14.36 13.51 ... 1.302 1.302 1.3
Attributes:
    case:        Anqing_202307
    grid:        CN3AH_135X138
    createtime:  2023-08-08

In [7]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in dataset.data_vars}
dataset.to_netcdf('D:/Download/AQ_2307_isam2.nc',encoding=encoding)